In [1]:
# import libraries
import torch
from torch import nn
from torch.nn import functional as F
import torchvision

import torchvision.transforms as transforms
#import torchvision.datasets as datasets
#from torch.utils.data import Dataset
#from torch.utils.data import DataLoader

In [2]:
import sys
from torch import nn
import torch
from datetime import timedelta
sys.path.append("../")
sys.path.append("../../")
import os
import random
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from open_data import open_data, create_global_batch
from dataset import RadarDataset,RadarCollate
from simple_model import LSTM
from train import train
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [3]:
PATH_CSV="/Users/iris/Documents/radar_deep/Radar_Traffic_Counts.csv" #"/Users/clemencevast/Documents/Courses/MachineLearning/RadarTrafficData_files/Radar_Traffic_Counts.csv" #
radar_name=' CAPITAL OF TEXAS HWY / LAKEWOOD DR'

In [4]:
df1=open_data(PATH_CSV, direction="NB", radar=radar_name, year=2018)

In [5]:
batch_df=create_global_batch(df1, window_x_day=6, window_label_day=1, gap_acquisition=1, tot_len_day=365)

Building batch 0 
 x begin 2018-01-02 label begin 2018-01-08 end period 2018-12-23 
We do not have all the dates for the time period in label , 96 94
Building batch 0 
 x begin 2018-01-03 label begin 2018-01-09 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 574
Building batch 0 
 x begin 2018-01-04 label begin 2018-01-10 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 574
Building batch 0 
 x begin 2018-01-05 label begin 2018-01-11 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 574
Building batch 0 
 x begin 2018-01-06 label begin 2018-01-12 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 574
Building batch 0 
 x begin 2018-01-07 label begin 2018-01-13 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 520
Building batch 0 
 x begin 2018-01-08 label begin 2018-01-14 end period 2018-12-23 
We do not have all the dat

We do not have all the dates for the time period in x , 576 572
Building batch 6 
 x begin 2018-03-05 label begin 2018-03-11 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 572
Building batch 6 
 x begin 2018-03-06 label begin 2018-03-12 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 568
Building batch 6 
 x begin 2018-03-07 label begin 2018-03-13 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 570
Building batch 6 
 x begin 2018-03-08 label begin 2018-03-14 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 570
Building batch 6 
 x begin 2018-03-09 label begin 2018-03-15 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 564
Building batch 6 
 x begin 2018-03-10 label begin 2018-03-16 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 564
Building batch 6 
 x begin 2018-03-11 label begi

We do not have all the dates for the time period in x , 576 574
Building batch 22 
 x begin 2018-06-03 label begin 2018-06-09 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 574
Building batch 22 
 x begin 2018-06-04 label begin 2018-06-10 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 574
Building batch 22 
 x begin 2018-06-05 label begin 2018-06-11 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 574
Building batch 22 
 x begin 2018-06-06 label begin 2018-06-12 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 574
Building batch 22 
 x begin 2018-06-07 label begin 2018-06-13 end period 2018-12-23 
We do not have all the dates for the time period in label , 96 95
Building batch 22 
 x begin 2018-06-08 label begin 2018-06-14 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 575
Building batch 22 
 x begin 2018-06-09 l

We do not have all the dates for the time period in x , 576 0
Building batch 57 
 x begin 2018-09-01 label begin 2018-09-07 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 3
Building batch 57 
 x begin 2018-09-02 label begin 2018-09-08 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 99
Building batch 57 
 x begin 2018-09-03 label begin 2018-09-09 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 195
Building batch 57 
 x begin 2018-09-04 label begin 2018-09-10 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 291
Building batch 57 
 x begin 2018-09-05 label begin 2018-09-11 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 387
Building batch 57 
 x begin 2018-09-06 label begin 2018-09-12 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 483
Building batch 57 
 x begin 2018-09-07 label be

 x begin 2018-11-03 label begin 2018-11-09 end period 2018-12-23 
Building batch 77 
 x begin 2018-11-04 label begin 2018-11-10 end period 2018-12-23 
Building batch 78 
 x begin 2018-11-05 label begin 2018-11-11 end period 2018-12-23 
Building batch 79 
 x begin 2018-11-06 label begin 2018-11-12 end period 2018-12-23 
Building batch 80 
 x begin 2018-11-07 label begin 2018-11-13 end period 2018-12-23 
Building batch 81 
 x begin 2018-11-08 label begin 2018-11-14 end period 2018-12-23 
We do not have all the dates for the time period in label , 96 94
Building batch 81 
 x begin 2018-11-09 label begin 2018-11-15 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 574
Building batch 81 
 x begin 2018-11-10 label begin 2018-11-16 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 574
Building batch 81 
 x begin 2018-11-11 label begin 2018-11-17 end period 2018-12-23 
We do not have all the dates for the time period in x , 576 

In [6]:
batch_df

,vol_data_x,vol_label_y
0,"[39, 24, 28, 20, 14, 14, 12, 8, 6, 7, 2, 10, 9...","[41, 37, 17, 26, 18, 17, 24, 11, 14, 7, 6, 15,..."
1,"[32, 27, 32, 25, 26, 15, 15, 18, 20, 7, 9, 15,...","[37, 34, 29, 29, 20, 15, 13, 8, 9, 14, 9, 12, ..."
2,"[63, 57, 57, 36, 32, 37, 29, 28, 13, 14, 17, 1...","[49, 33, 32, 31, 24, 20, 13, 11, 17, 8, 11, 10..."
3,"[78, 66, 45, 52, 52, 39, 40, 38, 32, 25, 17, 2...","[58, 57, 59, 39, 44, 42, 27, 29, 30, 15, 13, 1..."
4,"[35, 23, 23, 24, 14, 10, 24, 18, 9, 8, 10, 10,...","[70, 63, 36, 49, 31, 35, 24, 27, 18, 14, 20, 1..."
...,...,...
91,"[58, 28, 26, 33, 20, 23, 13, 15, 6, 7, 13, 5, ...","[38, 35, 29, 18, 11, 20, 11, 14, 9, 13, 6, 7, ..."
92,"[50, 40, 36, 22, 27, 22, 19, 27, 15, 12, 7, 9,...","[41, 42, 32, 32, 20, 13, 22, 15, 10, 8, 5, 11,..."
93,"[70, 63, 57, 46, 38, 33, 25, 30, 24, 16, 10, 1...","[63, 52, 39, 29, 25, 17, 19, 18, 10, 8, 6, 9, ..."
94,"[87, 78, 75, 57, 45, 31, 35, 28, 26, 15, 10, 1...","[82, 87, 67, 69, 46, 32, 40, 27, 23, 19, 23, 1..."


In [7]:
#batch_df["mean_x"]=batch_df["vol_data_x"].apply(lambda x : np.mean(x))
#batch_df["std_x"]=batch_df["vol_data_x"].apply(lambda x : np.std(x))
#batch_df["mean_y"]=batch_df["vol_label_y"].apply(lambda x : np.mean(x))

In [8]:
batch_df

,vol_data_x,vol_label_y
0,"[39, 24, 28, 20, 14, 14, 12, 8, 6, 7, 2, 10, 9...","[41, 37, 17, 26, 18, 17, 24, 11, 14, 7, 6, 15,..."
1,"[32, 27, 32, 25, 26, 15, 15, 18, 20, 7, 9, 15,...","[37, 34, 29, 29, 20, 15, 13, 8, 9, 14, 9, 12, ..."
2,"[63, 57, 57, 36, 32, 37, 29, 28, 13, 14, 17, 1...","[49, 33, 32, 31, 24, 20, 13, 11, 17, 8, 11, 10..."
3,"[78, 66, 45, 52, 52, 39, 40, 38, 32, 25, 17, 2...","[58, 57, 59, 39, 44, 42, 27, 29, 30, 15, 13, 1..."
4,"[35, 23, 23, 24, 14, 10, 24, 18, 9, 8, 10, 10,...","[70, 63, 36, 49, 31, 35, 24, 27, 18, 14, 20, 1..."
...,...,...
91,"[58, 28, 26, 33, 20, 23, 13, 15, 6, 7, 13, 5, ...","[38, 35, 29, 18, 11, 20, 11, 14, 9, 13, 6, 7, ..."
92,"[50, 40, 36, 22, 27, 22, 19, 27, 15, 12, 7, 9,...","[41, 42, 32, 32, 20, 13, 22, 15, 10, 8, 5, 11,..."
93,"[70, 63, 57, 46, 38, 33, 25, 30, 24, 16, 10, 1...","[63, 52, 39, 29, 25, 17, 19, 18, 10, 8, 6, 9, ..."
94,"[87, 78, 75, 57, 45, 31, 35, 28, 26, 15, 10, 1...","[82, 87, 67, 69, 46, 32, 40, 27, 23, 19, 23, 1..."


In [9]:
def get_df_stats(df,columns=None):
    """ return mean and std for the columns"""
    if columns is None: 
        columns=["vol_data_x","vol_label_y"]
    dic={}
    global_mean=0
    global_std=0
    for name in columns:
        df["{}_min".format(name)]=df[name].apply(lambda x : np.min(x))
        df["{}_max".format(name)]=df[name].apply(lambda x : np.max(x))
        global_mean+=df["{}_min".format(name)].min()
        global_std+=df["{}_max".format(name)].max()
    return global_mean/len(columns),global_std/len(columns)
    
def apply_norm(df,mini,maxi,columns=None):
    """ apply std"""
    if columns is None: 
        columns=["vol_data_x","vol_label_y"]
    for name in columns:
        df["{}_norm".format(name)]=df[name].apply(lambda x : (x-mini)/(maxi-mini))
    return df

In [10]:
df_mean,df_std=get_df_stats(batch_df)
new_data=apply_norm(batch_df,df_mean,df_std)


In [11]:
get_df_stats(new_data,["vol_data_x_norm","vol_label_y_norm"])

(0.0, 1.0)

In [12]:
new_data

,vol_data_x,vol_label_y,vol_data_x_min,vol_data_x_max,vol_label_y_min,vol_label_y_max,vol_data_x_norm,vol_label_y_norm,vol_data_x_norm_min,vol_data_x_norm_max,vol_label_y_norm_min,vol_label_y_norm_max
0,"[39, 24, 28, 20, 14, 14, 12, 8, 6, 7, 2, 10, 9...","[41, 37, 17, 26, 18, 17, 24, 11, 14, 7, 6, 15,...",2,496,6,482,"[0.03927813163481953, 0.02335456475583864, 0.0...","[0.041401273885350316, 0.037154989384288746, 0...",0.000000,0.524416,0.004246,0.509554
1,"[32, 27, 32, 25, 26, 15, 15, 18, 20, 7, 9, 15,...","[37, 34, 29, 29, 20, 15, 13, 8, 9, 14, 9, 12, ...",3,496,4,482,"[0.03184713375796178, 0.02653927813163482, 0.0...","[0.037154989384288746, 0.03397027600849257, 0....",0.001062,0.524416,0.002123,0.509554
2,"[63, 57, 57, 36, 32, 37, 29, 28, 13, 14, 17, 1...","[49, 33, 32, 31, 24, 20, 13, 11, 17, 8, 11, 10...",3,496,6,500,"[0.06475583864118896, 0.058386411889596604, 0....","[0.049893842887473464, 0.032908704883227176, 0...",0.001062,0.524416,0.004246,0.528662
3,"[78, 66, 45, 52, 52, 39, 40, 38, 32, 25, 17, 2...","[58, 57, 59, 39, 44, 42, 27, 29, 30, 15, 13, 1...",3,500,7,434,"[0.08067940552016985, 0.06794055201698514, 0.0...","[0.059447983014861996, 0.058386411889596604, 0...",0.001062,0.528662,0.005308,0.458599
4,"[35, 23, 23, 24, 14, 10, 24, 18, 9, 8, 10, 10,...","[70, 63, 36, 49, 31, 35, 24, 27, 18, 14, 20, 1...",3,500,6,402,"[0.03503184713375796, 0.022292993630573247, 0....","[0.07218683651804671, 0.06475583864118896, 0.0...",0.001062,0.528662,0.004246,0.424628
...,...,...,...,...,...,...,...,...,...,...,...,...
91,"[58, 28, 26, 33, 20, 23, 13, 15, 6, 7, 13, 5, ...","[38, 35, 29, 18, 11, 20, 11, 14, 9, 13, 6, 7, ...",3,488,4,484,"[0.059447983014861996, 0.027600849256900213, 0...","[0.03821656050955414, 0.03503184713375796, 0.0...",0.001062,0.515924,0.002123,0.511677
92,"[50, 40, 36, 22, 27, 22, 19, 27, 15, 12, 7, 9,...","[41, 42, 32, 32, 20, 13, 22, 15, 10, 8, 5, 11,...",3,488,5,501,"[0.050955414012738856, 0.040339702760084924, 0...","[0.041401273885350316, 0.04246284501061571, 0....",0.001062,0.515924,0.003185,0.529724
93,"[70, 63, 57, 46, 38, 33, 25, 30, 24, 16, 10, 1...","[63, 52, 39, 29, 25, 17, 19, 18, 10, 8, 6, 9, ...",3,501,6,480,"[0.07218683651804671, 0.06475583864118896, 0.0...","[0.06475583864118896, 0.05307855626326964, 0.0...",0.001062,0.529724,0.004246,0.507431
94,"[87, 78, 75, 57, 45, 31, 35, 28, 26, 15, 10, 1...","[82, 87, 67, 69, 46, 32, 40, 27, 23, 19, 23, 1...",3,501,5,409,"[0.09023354564755838, 0.08067940552016985, 0.0...","[0.08492569002123142, 0.09023354564755838, 0.0...",0.001062,0.529724,0.003185,0.432059


In [13]:
#get_df_stats(new_data,columns=["vol_data_x_norm","vol_label_y_norm"])

In [14]:
# sample_rate = 16000
batch_size = 1

In [15]:
normalize=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[df_mean],
                         std=[df_std])
])
fullset = RadarDataset(dataframe=batch_df,transform=None)
print(type(fullset))
ptrain,pval,ptest=0.8,0.05,0.15
trainglobdataset, testdataset = train_test_split(fullset, test_size=ptest)
traindataset, valdataset = train_test_split(trainglobdataset, test_size=pval/(1-ptest))

<class 'dataset.RadarDataset'>
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)
(96,)


In [16]:
collate_fn = RadarCollate()
trainloader = DataLoader(traindataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
valloader = DataLoader(valdataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
testloader = DataLoader(testdataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=False)

In [17]:
c= 0
for x in trainloader:
    print("batch number {}".format(c))
    print(x[0].shape, x[1].shape, "\n")
    #print(x)
    c+=1

batch number 0
torch.Size([1, 576]) torch.Size([1, 96]) 

batch number 1
torch.Size([1, 576]) torch.Size([1, 96]) 

batch number 2
torch.Size([1, 576]) torch.Size([1, 96]) 

batch number 3
torch.Size([1, 576]) torch.Size([1, 96]) 

batch number 4
torch.Size([1, 576]) torch.Size([1, 96]) 

batch number 5
torch.Size([1, 576]) torch.Size([1, 96]) 

batch number 6
torch.Size([1, 576]) torch.Size([1, 96]) 

batch number 7
torch.Size([1, 576]) torch.Size([1, 96]) 

batch number 8
torch.Size([1, 576]) torch.Size([1, 96]) 

batch number 9
torch.Size([1, 576]) torch.Size([1, 96]) 

batch number 10
torch.Size([1, 576]) torch.Size([1, 96]) 

batch number 11
torch.Size([1, 576]) torch.Size([1, 96]) 

batch number 12
torch.Size([1, 576]) torch.Size([1, 96]) 

batch number 13
torch.Size([1, 576]) torch.Size([1, 96]) 

batch number 14
torch.Size([1, 576]) torch.Size([1, 96]) 

batch number 15
torch.Size([1, 576]) torch.Size([1, 96]) 

batch number 16
torch.Size([1, 576]) torch.Size([1, 96]) 

batch n

In [18]:
input_dim = 576    
hidden_dim = 256
layer_dim = 2
output_dim = 96
seq_dim = 1

lr = 0.05
n_epochs = 200
iterations_per_epoch = len(trainloader)
best_acc = 0

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
model=LSTM()


In [19]:
iteration,loss_train_list,loss_val_list=train(model, trainloader, valloader, lr=lr, n_epochs=n_epochs, device=device)

/Users/iris/opt/anaconda3/envs/dl_env/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([1, 96])) that is different to the input size (torch.Size([96])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 1 loss train 0.0005064644501544535 loss val 0.0023786858655512333
epoch 11 loss train 4.456012092646233e-08 loss val 0.00016950529243331403
epoch 21 loss train 9.410598067915998e-06 loss val 0.00010987396672135219
epoch 31 loss train 4.9918572386786764e-08 loss val 9.735547791933641e-05
epoch 41 loss train 2.414771870462573e-06 loss val 5.1223894843133166e-05
epoch 51 loss train 9.267682798963506e-06 loss val 9.751554171089083e-05
epoch 61 loss train 1.491485051019481e-07 loss val 4.293857637094334e-05
epoch 71 loss train 7.108787940524053e-07 loss val 2.9098173399688676e-05
epoch 81 loss train 3.783165425375046e-08 loss val 1.0239556104352232e-05
epoch 91 loss train 1.5328052427321381e-07 loss val 5.926535777689423e-06


([1, 11, 21, 31, 41, 51, 61, 71, 81, 91],
 [0.0005064644501544535,
  4.456012092646233e-08,
  9.410598067915998e-06,
  4.9918572386786764e-08,
  2.414771870462573e-06,
  9.267682798963506e-06,
  1.491485051019481e-07,
  7.108787940524053e-07,
  3.783165425375046e-08,
  1.5328052427321381e-07],
 [0.0023786859,
  0.00016950529,
  0.00010987397,
  9.735548e-05,
  5.1223895e-05,
  9.751554e-05,
  4.2938576e-05,
  2.9098173e-05,
  1.0239556e-05,
  5.926536e-06])